###  MicroGrad demo

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mycrograd_debug.engine_debug import Value
from mycrograd_debug.nn_debug import Neuron, Layer, MLP
from mycrograd_debug.drawviz_debug import draw_dot,draw_nn,print_all_values
import pprint
np.random.seed(1337)
random.seed(1337)
number_of_samples=2
number_of_iterations=2
pp = pprint.PrettyPrinter(indent=4)

In [2]:
# make up a dataset

from sklearn.datasets import make_moons, make_blobs
X, y = make_moons(n_samples=number_of_samples, noise=0.1)
pp.pprint(X)
pp.pprint(y)
y = y*2 - 1 # make y be -1 or 1
pp.pprint(y)

array([[ 0.8572558 ,  0.03006228],
       [-0.05120403,  0.50473341]])
array([0, 1])
array([-1,  1])


In [3]:
# initialize a model 
# model = MLP(2, [16, 16, 1]) # 2-layer neural network
model = MLP(2, [2, 1], debug_bw=False) # 2-layer neural network
# print(model)
# pp.pprint(model.parameters())
Xb, yb = X, y
inputs = [list(map(Value, xrow)) for xrow in Xb]
pp.pprint(inputs)
for i in range(len(inputs)):
    for j in range(len(inputs[i])):
        inputs[i][j].type = "i"
pp.pprint(inputs)

Module nn MLP: structure [2, 2, 1]
[   [   Value(name=v010,layernumber=,neuronnumber=,weightnumber=,type=,data=0.8572557995669348, grad=0),
        Value(name=v011,layernumber=,neuronnumber=,weightnumber=,type=,data=0.03006227935826109, grad=0)],
    [   Value(name=v012,layernumber=,neuronnumber=,weightnumber=,type=,data=-0.051204029028939925, grad=0),
        Value(name=v013,layernumber=,neuronnumber=,weightnumber=,type=,data=0.5047334093296381, grad=0)]]
[   [   Value(name=v010,layernumber=,neuronnumber=,weightnumber=,type=i,data=0.8572557995669348, grad=0),
        Value(name=v011,layernumber=,neuronnumber=,weightnumber=,type=i,data=0.03006227935826109, grad=0)],
    [   Value(name=v012,layernumber=,neuronnumber=,weightnumber=,type=i,data=-0.051204029028939925, grad=0),
        Value(name=v013,layernumber=,neuronnumber=,weightnumber=,type=i,data=0.5047334093296381, grad=0)]]


In [4]:
# loss function
def loss():
    
    # forward the model to get scores
    scores = list(map(model, inputs))
    
    # svm "max-margin" loss
    losses = [(1 + -yi*scorei).relu() for yi, scorei in zip(yb, scores)]
    data_loss = sum(losses) * (1.0 / len(losses))
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    total_loss = data_loss + reg_loss
    
    # also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(yb, scores)]
    return total_loss, sum(accuracy) / len(accuracy)

# total_loss, acc = loss()
# print(total_loss, acc)

In [5]:


# optimization
for k in range(number_of_iterations):
    print("start step %d" %k)
    # pp.pprint(model.parameters())

    # forward
    total_loss, acc = loss()
    print("step %d loss calc" %k)
    print_all_values(total_loss)
    dot=draw_dot(total_loss)
    dot.render("images/opt_01_step%d_1loss" % k , format="svg", view=True)

    # backward
    model.zero_grad()
    print("step %d zero grad" %k)
    print_all_values(total_loss)
    dot=draw_dot(total_loss)
    dot.render("images/opt_01_step%d_2zero" % k , format="svg", view=True)

    total_loss.backward()
    print("step %d backward" %k)
    print_all_values(total_loss)
    dot=draw_dot(total_loss)
    dot.render("images/opt_01_step%d_3back" % k , format="svg", view=True)
    
    # update (sgd)
    learning_rate = 1.0 - 0.9*k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad
    
    # if k % 1 == 0:
    #     print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")
    print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")


start step 0
step 0 loss calc
name ty   data   grad
v001 w1   0.24   0.00
v002 w2   0.07   0.00
v003  b   0.00   0.00
v004 w1  -0.27   0.00
v005 w2   0.17   0.00
v006  b   0.00   0.00
v007 w1  -0.67   0.00
v008 w2   0.65   0.00
v009  b   0.00   0.00
v010  i   0.86   0.00
v011  i   0.03   0.00
v012  i  -0.05   0.00
v013  i   0.50   0.00
v014      0.20   0.00
v015      0.20   0.00
v016      0.00   0.00
v017  a   0.20   0.00
v018      0.20   0.00
v019     -0.23   0.00
v020     -0.23   0.00
v021      0.01   0.00
v022  a  -0.22   0.00
v023      0.00   0.00
v024     -0.14   0.00
v025     -0.14   0.00
v026      0.00   0.00
v027  a  -0.14   0.00
v028     -0.01   0.00
v029     -0.01   0.00
v030      0.03   0.00
v031  a   0.02   0.00
v032      0.02   0.00
v033      0.01   0.00
v034      0.01   0.00
v035      0.09   0.00
v036  a   0.10   0.00
v037      0.10   0.00
v038     -0.01   0.00
v039     -0.01   0.00
v040      0.07   0.00
v041  a   0.05   0.00
v042      1.00   0.00
v043     -0.14   0.00
v0

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from message bus without replying
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/curren

step 1 zero grad
name ty   data   grad
v001 w1   0.54   0.00
v002 w2  -0.09   0.00
v003  b   0.00   0.00
v004 w1  -0.28   0.00
v005 w2   0.34   0.00
v006  b   0.32   0.00
v007 w1  -0.76   0.00
v008 w2   0.70   0.00
v009  b   0.00   0.00
v010  i   0.86  -0.08
v011  i   0.03  -0.02
v012  i  -0.05   0.17
v013  i   0.50  -0.03
v079      0.46   0.00
v080      0.46   0.00
v081     -0.00   0.00
v082  a   0.46   0.00
v083      0.46   0.00
v084     -0.24   0.00
v085      0.08   0.00
v086      0.01   0.00
v087  a   0.09   0.00
v088      0.09   0.00
v089     -0.35   0.00
v090     -0.35   0.00
v091      0.06   0.00
v092  a  -0.29   0.00
v093     -0.03   0.00
v094     -0.03   0.00
v095     -0.05   0.00
v096  a  -0.07   0.00
v097      0.00   0.00
v098      0.01   0.00
v099      0.34   0.00
v100      0.17   0.00
v101  a   0.51   0.00
v102      0.51   0.00
v103     -0.00   0.00
v104      0.00   0.00
v105      0.36   0.00
v106  a   0.36   0.00
v107      1.00   0.00
v108     -0.29   0.00
v109      1.00 

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.ServiceUnknown: The name org.gnome.eog was not provided by any .service files


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
